In [1]:
# Install the needed libraries
import pandas as pd
import numpy as np
import time
import warnings
from pandas.core.common import SettingWithCopyWarning
import yaml
import cvxpy as cp
from datetime import datetime
import json

D:\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
D:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Ignore SettingWithCopyWarning 
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [3]:
def load_config():
    try:
        with open('config_file.yaml') as f:
            global constants
            constants = yaml.safe_load(f)
    except OSError as e:
        raise e
    except Exception as e:
        raise e

In [4]:
load_config()

In [5]:
# Import the data
# File name is specified in config file
data_input = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='data', engine='pyxlsb')
real_input = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='Real', engine='pyxlsb')
bandwidth_input = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='PS', engine='pyxlsb')
match_input = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='match', engine='pyxlsb')

In [6]:
# Copy datasets to implement further changes
data = data_input.copy()
real = real_input.copy()
bandwidth = bandwidth_input.copy()
match = match_input.copy()

In [7]:
# Create a dict of names on a spreadsheets that we are going to use
names = {"Тариф" : "cost",
        "Плечо" : "distance",
        "Дата" : "date",
        "НБ" : "origin",
        "ОУ" : "point",
        "НП" : "brand",
        "НП_ПС" : "product",
        "Объем" : "volume",
        "reg": "region"} 

In [8]:
# Rename the names in dataframes
def rename_columns(df_list, names):
    for df in df_list:
         df.rename(columns=names, inplace=True)
    return df_list

In [9]:
data, real, bandwidth, match = rename_columns([data, real, bandwidth, match], names)

In [10]:
# Check for duplicated; will raise an error if there are duplicates
assert data[data.duplicated(subset = ['point', 'origin', 'region', 'brand', 'date'])].empty == True
assert real[real.duplicated(subset = ['point', 'brand', 'date'])].empty == True
assert bandwidth[bandwidth.duplicated(subset = ['origin', 'product', 'date'])].empty == True

In [11]:
# Make subset for the concrete time period
def select_time_period(df):
    df_subset = df[df['date'] == constants['MONTH']]
    return df_subset

# Make subset for the concrete region - ONLY FOR EXPERIMENTS, IN THE FINAL VERSION ALL REGIONS GO TOGETHER
def select_region(df):
    regions_to_select = []
    
    for region_number in constants['REGION']:
        region_name = 'Регион ' + str(region_number)
        regions_to_select.append(region_name)
        
    if 'region' in df.columns:
        df = df[(df['region'].isin(regions_to_select))]
    return df
    
# Adjust datatypes. Enumerate oilbases and petrol stations      
def adjust_datatypes(df_list):
    df_new = []
    number = 0
    oilbases_dict = dict()

    for i in bandwidth['origin'].unique():
        oilbases_dict[i] = number
        number+=1
        
    for df in df_list:
        if 'origin' in df.columns:
            df = df.replace({"origin": oilbases_dict})
        if 'point' in df.columns:
            df['point'] = df['point'].map(lambda x: int(x.strip('АЗС ')))
        df_new.append(df)
        
    return df_new

In [12]:
# Apply changes to dataframes
def apply_changes_dfs(data, real, bandwidth, match):
    
    data_subset = select_time_period(data)
    real_subset = select_time_period(real)
    bandwidth_subset = select_time_period(bandwidth)

    data_subset = select_region(data_subset)

    data_subset, real_subset, bandwidth_subset = adjust_datatypes([data_subset, real_subset, bandwidth_subset])
    data_subset['cost'] = data_subset['cost'].round(4)
    
    return data_subset, real_subset, bandwidth_subset

In [13]:
data_subset, real_subset, bandwidth_subset = apply_changes_dfs(data, real, bandwidth, match)

In [14]:
# Create brand-product dict
def create_brand_product_dict(df):
    match_brand_product = dict(zip(df['brand'], df['product']))
    
    match_product_brand = dict()
    for key, value in match_brand_product.items():
        match_product_brand.setdefault(value, list()).append(key)
    return match_brand_product, match_product_brand

# Create numeration dict and inverted dict
def create_numeration_dict(df, parameter):
    match_dict = dict(enumerate(df[parameter]))
    inv_dict = {v: k for k, v in match_dict.items()}
    return match_dict, inv_dict

# Create numeration dict and inverted dict
def create_prod_numeration_dict(df, parameter):
    match_dict = dict(enumerate(df[parameter].unique()))
    inv_dict = {v: k for k, v in match_dict.items()}
    return match_dict, inv_dict

In [15]:
# Create 6 dictionaries (brand-product, brand-number, product-number, and inverted)
match_brand_product, match_product_brand = create_brand_product_dict(match)
match_brand_number, match_number_brand = create_numeration_dict(match,'brand')
match_product_number, match_number_product = create_prod_numeration_dict(match,'product')

In [16]:
# Create name of variable
def create_var_Name(row):
    var_name = 'P' + str(int(row["point"])) + 'O' + str(int(row["origin"])) +  '_brand_' + str(int(row["brand_number"]))
    return var_name

# Reorganize the data. Select only specified columns, enumerate brands and products
def reorganize_df(df, columns_to_keep, match_number_brand, match_number_product):
    df_reorganized = df.reset_index().loc[:, columns_to_keep]
    df_reorganized['brand_number'] = df_reorganized['brand'].map(lambda x: match_number_brand.get(x))
    df_reorganized['product_number'] = df_reorganized['product'].map(lambda x: match_number_product.get(x))
    
    return df_reorganized

In [17]:
columns_to_keep = ['origin', 'point', 'brand', 'date', 'product', 'cost', 'region']
df_reorganized = reorganize_df(data_subset, columns_to_keep, match_number_brand, match_number_product)

# Create a name variable
df_reorganized['var_Name'] = df_reorganized.apply(lambda row: create_var_Name(row), axis=1)  

In [18]:
# Create demand and supply dataframes
# Demand - volume of each product needed to be transported to the point
# Supply - volume of each product available on the origin

def create_demand_supply_dfs(real, bandwidth):
    demand = real[['point', 'brand', 'product', 'volume', 'date']]
    demand.rename(columns={'volume':'demand'}, inplace=True)
    demand['brand_number'] = demand['brand'].map(lambda x: match_number_brand.get(x))
    demand['product_number'] = demand['product'].map(lambda x: match_number_product.get(x))
    demand['demand'] = demand['demand'].round(4)
    
    supply = bandwidth[['origin', 'product', 'volume', 'date']]
    supply.rename(columns={'volume':'supply'}, inplace=True)
    supply['product_number'] = supply['product'].map(lambda x: match_number_product.get(x))
    supply['supply'] = supply['supply'].round(4)
    
    return demand, supply

In [19]:
demand, supply = create_demand_supply_dfs(real_subset, bandwidth_subset)

In [20]:
# Limit demand and supply with only needed origins and points
supply = supply[supply['origin'].isin(df_reorganized.origin.unique())]
demand = demand[demand['point'].isin(df_reorganized.point.unique())]

In [21]:
# Make a copy
df_reorganized_trans = df_reorganized.copy()

# Check for where ALL volumes equals == 0 
df_trans= supply[supply['supply'] <= 0][['origin', 'product_number']]
origins_zero_volume = set(zip(df_trans.origin, df_trans.product_number))

pairs_origins_products_to_delete = []
pairs_points_brands_to_delete = []

# Filter df_reogranized
for _, row in df_reorganized.iterrows():

    origin, point, product, brand = row['origin'], row['point'], row['product_number'], row['brand_number']
    
    if supply[(supply['origin'] == origin) & (supply['product_number'] == product)].empty:
        pairs_origins_products_to_delete.append((origin, product))

    if demand[(demand['point'] == point) & (demand['brand_number'] == brand)].empty:
        pairs_points_brands_to_delete.append((point, brand))

pairs_origins_products_to_delete = set(pairs_origins_products_to_delete)
pairs_points_brands_to_delete = set(pairs_points_brands_to_delete)

# Update origins to delete
pairs_origins_products_to_delete.update(origins_zero_volume)

df_reorganized_trans['origin_product'] = list(zip(df_reorganized_trans.origin, df_reorganized_trans.product_number))
df_reorganized_trans['point_brand'] = list(zip(df_reorganized_trans.point, df_reorganized_trans.brand_number))
  
df_reorganized_trans = df_reorganized_trans.drop(df_reorganized_trans[df_reorganized_trans['origin_product'].isin(pairs_origins_products_to_delete)].index)
df_reorganized_trans = df_reorganized_trans.drop(df_reorganized_trans[df_reorganized_trans['point_brand'].isin(pairs_points_brands_to_delete)].index)
   
        
# Reassign df
df_reorganized = df_reorganized_trans
df_reorganized = df_reorganized.reset_index().drop('index', axis=1)

In [22]:
df_reorganized.shape

(53802, 12)

# CVXPY

In [23]:
brands = df_reorganized['brand_number'].unique()

In [24]:
brand_info_map = {}

def make_brand(brand_id, name):
    return {'name': name, 'id': brand_id, 'product_transportations': {}}

In [25]:
def make_transportation(origin, point, cost, var_name):
    return {'origin': origin, 'point': point, 'cost': cost, 'var_name': var_name}

In [26]:
def add_transportation_to_brand(brand, product_id, transportation):
    if brand['product_transportations'].get(product_id) == None:
        brand['product_transportations'][product_id] = [transportation]
        return brand
    
    brand['product_transportations'].get(product_id).append(transportation)

    return brand

In [27]:
def append_row_to_map(brand_map, row, index):
    brand_id = row['brand_number']
    
    if brand_map.get(brand_id) == None:
        brand_map[brand_id] = make_brand(brand_id, row['brand'])
        
    brand_item = brand_map.get(brand_id)
    product_id = row['product_number']
    
    transportation = make_transportation(row['origin'], row['point'], row['cost'], index);
    brand_map[brand_id] = add_transportation_to_brand(brand_item, product_id, transportation)
    
    return brand_map

In [28]:
for i, row in df_reorganized.iterrows():
    brand_info_map = append_row_to_map(brand_info_map, row, i)

json_formatted_str = json.dumps(brand_info_map, indent=2, ensure_ascii=False)

In [29]:
# Make dicts for groups - either D (diesel) or P (petrol)
def make_match_dicts(match_brand_product):
    
    match_brand_group = dict()
    match_product_group = dict()

    for k,v in match_brand_product.items():
        if "ДТ" in v:
            match_brand_group[k] = "D"
            match_product_group[v] = "D"
        
        elif ("АИ" in v) | ("Аи" in v):
            match_brand_group[k] = "P"
            match_product_group[v] = "P"

    match_group_brand = dict()

    for k,v in match_brand_group.items():
        match_group_brand.setdefault(v, list()).append(k)
    
    
    match_group_product = dict()

    for k,v in match_product_group.items():
        match_group_product.setdefault(v, list()).append(k)
    
    return match_brand_group, match_product_group, match_group_brand, match_group_product

In [30]:
match_brand_group, match_product_group, match_group_brand, match_group_product = make_match_dicts(match_brand_product)

In [31]:
# Function for dividing supply between different products
def merge_supply_by_product(brand_number, match_brand_product, match_brand_number, match_product_brand, match_number_brand):
    """This function returns a set of brands' numbers that are produced from the same product"""
    brand_name = match_brand_number[brand_number]
    product_name = match_brand_product[brand_name]
    brands_overlap_names = match_product_brand[product_name]
    brands_overlap_numbers = set(match_number_brand.get(item) for item in brands_overlap_names)

    return brands_overlap_numbers

In [32]:
# Function for dividing supply between different groups
def merge_supply_by_group(brand_number, match_brand_number, match_number_brand, match_brand_group, match_group_brand):
    """This function returns a set of brands' numbers that are produced from the same product group - P or D"""
    brand_name = match_brand_number[brand_number]
    group_name = match_brand_group[brand_name]
    brands_overlap_names = match_group_brand[group_name]
    brands_overlap_numbers = set(match_number_brand.get(item) for item in brands_overlap_names)

    return brands_overlap_numbers

In [33]:
# Function for dividing supply between different brands
def merge_supply(brand_number, match_brand_product, match_brand_number, match_product_brand, match_number_brand):
    """This function returns a set of brands' numbers that are produced from the same product"""
    brand_name = match_brand_number[brand_number]
    product_name = match_brand_product[brand_name]
    brands_overlap_names = match_product_brand[product_name]
    brands_overlap_numbers = set(match_number_brand.get(item) for item in brands_overlap_names)

    return brands_overlap_numbers

In [34]:
# Get the list of unique points inside each brand
def extract_unique_points_of_brand(brand):
    points = []
    for product_id in brand.get('product_transportations').keys():
        for transportation in brand.get('product_transportations').get(product_id):
            points.append(transportation.get('point'))

    points = list(set(points))
    
    return points

# Get the list of unique origins inside each brand
def extract_unique_origins_of_brand(brand):
    origins = []
    for transportations in brand.get('product_transportations').values():
        for transportation in transportations:
            origins.append(transportation.get('origin'))

    origins = list(set(origins))
    
    return origins

# Extract the transportation of products from map dictionary
def get_brand_transportations(brand):
    brand_transpotrations = []
    for product_id in brand.get('product_transportations').keys():
        for transportation in brand.get('product_transportations').get(product_id):
            brand_transpotrations.append(transportation)
                
    return brand_transpotrations

# Get the list of unique origins (overall)
def extract_unique_origins(brand_map):
    origins = []
    for brand in brand_map.values():
        origins = origins + extract_unique_origins_of_brand(brand)

    return list(set(origins))

# Get the list of unique points (overall)
def extract_unique_points(brand_map):
    points = []
    for brand in brand_map.values():
        points = points + extract_unique_points_of_brand(brand)
        
    return list(set(points))

# Get transportations of concrete brand
def get_transportations(brand_map):
    transportations = []
    
    for brand in brand_map.values():
        transportations = transportations + get_brand_transportations(brand)
        
    return transportations

In [35]:
def define_problem(df):
    x = cp.Variable(len(df), nonneg = True)
    c = np.array(df['cost'])
    objective = cp.Minimize(c.T @ x)
    
    return x, objective

#### Define restriction #1: demand equals const

In [36]:
def make_restriction_demand(record, brands, x, demand):
    n_constraint = 1
    constraints = []
    
    for brand_id in brands:
        brand = record.get(brand_id);
        points = extract_unique_points_of_brand(brand)
    
        for p in points:
            transportations = get_brand_transportations(brand)
            brand_point_transpotrations = [transportation for transportation in transportations if transportation.get('point') == p]
            
            const = float(demand[(demand['point'] == p) & (demand['brand_number'] == brand_id)]['demand'])
            transportations_x = [x[transportation.get('var_name')] for transportation in brand_point_transpotrations]
            constraints.append(cp.sum(transportations_x) == const)
            n_constraint +=1
        
    return constraints, n_constraint, x

#### Define restriction #2: supply less or equal than const;  restriction #4: supply greater or equal to const * percent_loading

In [37]:
# Make restriction on the supply (concrete volume of product available on origin)
# Limitations by month, and by group S, D - upper bound
def make_restriction_supply_by_group_upper(constraints, record, bandwidth, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint):
    # Set for storing already processed brands 
    brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    #origins_with_inf_supply = list(bandwidth[bandwidth['volume'] == np.inf]['origin'].unique())
    
    for b in brands:
        if b in brands_accounted_for:
            continue
        
        brands_overlap_numbers = merge_supply_by_group(b, match_brand_number, match_number_brand, match_brand_group, match_group_brand)
        brands_accounted_for.update(brands_overlap_numbers)

        brands_list = [brand for brand in record.values() if brand.get('id') in brands_overlap_numbers]

        origins = []
        for brand in brands_list:
            origins += extract_unique_origins_of_brand(brand)
            
        origins = list(set(origins))
        
        transportations_list = []
        for brand in brands_list:
            transportations_list += get_brand_transportations(brand)

        for o in origins:
            
            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]
            product_name = match_brand_product[match_brand_number[b]]
            group_name = match_product_group[product_name]
            products_share_group = match_group_product[group_name]
            const = float(supply[(supply['origin'] == o) & (supply['product'].isin(products_share_group))]['supply'].sum())
            
            # Add a restriction on supply <= upper_const
            transportation_x = [x[transportation.get("var_name")] for transportation in origin_transportations]
            constraints.append(cp.sum(transportation_x) <= const)
            n_constraint +=1  
                
    return constraints, n_constraint, x

In [38]:
# Make restriction on supply (restriction on lower percentage of loading)
# Limitation by month, group D,and S - only lower bound
def make_restriction_supply_by_group_lower(constraints, record, bandwidth, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint):
    # Set for storing already processed brands 
    brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    number = 0
    oilbases_dict = dict()

    for i in bandwidth['origin'].unique():
        oilbases_dict[i] = number
        number+=1
        
    origins_with_inf_supply = set(oilbases_dict.values()).difference(constants['ORIGINS_TO_CONTROL'])
    
    for b in brands:
        if b in brands_accounted_for:
            continue
        
        brands_overlap_numbers = merge_supply_by_group(b, match_brand_number, match_number_brand, match_brand_group, match_group_brand)
        brands_accounted_for.update(brands_overlap_numbers)

        brands_list = [brand for brand in record.values() if brand.get('id') in brands_overlap_numbers]

        origins = []
        for brand in brands_list:
            origins += extract_unique_origins_of_brand(brand)
            
        origins = list(set(origins))
        
        transportations_list = []
        for brand in brands_list:
            transportations_list += get_brand_transportations(brand)

        for o in origins:
            
            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]

            product_name = match_brand_product[match_brand_number[b]]
            group_name = match_product_group[product_name]
            products_share_group = match_group_product[group_name]
            const = float(supply[(supply['origin'] == o) & (supply['product'].isin(products_share_group))]['supply'].sum())            
            
            if o in origins_with_inf_supply:
                continue       

            else:

                # Add a restriction on supply >= lower_const
                const *= constants['PERCENT_LOADING']
                transportation_x = [x[transportation.get("var_name")] for transportation in origin_transportations]
                constraints.append(cp.sum(transportation_x) >= const)
                n_constraint +=1 
                    
    return constraints, n_constraint, x

In [39]:
# Limitation by month, group S, D - upper + lower
def make_restriction_supply_by_group(constraints, record, bandwidth, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint):
    # Set for storing already processed brands 
    brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    #origins_with_inf_supply = list(bandwidth[bandwidth['volume'] == np.inf]['origin'].unique())
    
    for b in brands:
        if b in brands_accounted_for:
            continue
        
        brands_overlap_numbers = merge_supply_by_group(b, match_brand_number, match_number_brand, match_brand_group, match_group_brand)
        brands_accounted_for.update(brands_overlap_numbers)

        brands_list = [brand for brand in record.values() if brand.get('id') in brands_overlap_numbers]

        origins = []
        for brand in brands_list:
            origins += extract_unique_origins_of_brand(brand)
            
        origins = list(set(origins))
        
        transportations_list = []
        for brand in brands_list:
            transportations_list += get_brand_transportations(brand)

        for o in origins:
            
            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]

            product_name = match_brand_product[match_brand_number[b]]
            group_name = match_product_group[product_name]
            products_share_group = match_group_product[group_name]
            const = float(supply[(supply['origin'] == o) & (supply['product'].isin(products_share_group))]['supply'].sum())
            
            
            if o in origins_with_inf_supply:
                # Add a restriction on supply <= upper_const
                transportation_x = [x[transportation.get("var_name")] for transportation in origin_transportations]
                constraints.append(cp.sum(transportation_x) <= const)
                n_constraint +=1           

            else:
                # Add a restriction on supply <= upper_const
                transportation_x = [x[transportation.get("var_name")] for transportation in origin_transportations]
                constraints.append(cp.sum(transportation_x) <= const)
                n_constraint +=1            

                # Add a restriction on supply >= lower_const

                const *= constants['PERCENT_LOADING']
                transportation_x = [x[transportation.get("var_name")] for transportation in origin_transportations]
                constraints.append(cp.sum(transportation_x) >= const)
                n_constraint +=1 
                   
    return constraints, n_constraint, x

In [40]:
# Limitation by month and product - upper bound
def make_restriction_supply_by_product_upper(constraints, record, bandwidth, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint):
    # Set for storing already processed brands 
    brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    #origins_with_inf_supply = list(bandwidth[bandwidth['volume'] == np.inf]['origin'].unique())
    
    for b in brands:
        if b in brands_accounted_for:
            continue
        
        brands_overlap_numbers = merge_supply_by_product(b, match_brand_product, match_brand_number, match_product_brand, match_number_brand)
        brands_accounted_for.update(brands_overlap_numbers)

        brands_list = [brand for brand in record.values() if brand.get('id') in brands_overlap_numbers]

        origins = []
        for brand in brands_list:
            origins += extract_unique_origins_of_brand(brand)
            
        origins = list(set(origins))
        
        transportations_list = []
        for brand in brands_list:
            transportations_list += get_brand_transportations(brand)
            
            
        for o in origins:
            
            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]
            product_name = match_brand_product[match_brand_number[b]]
            const = float(supply[(supply['origin'] == o) & (supply['product'] == product_name)]['supply'])
            
            # Add a restriction on supply <= upper_const
            transportation_x = [x[transportation.get("var_name")] for transportation in origin_transportations]
            constraints.append(cp.sum(transportation_x) <= const)
            n_constraint +=1 
            
    return constraints, n_constraint, x

In [41]:
# Limitation by month and product - lower bound
def make_restriction_supply_by_product_lower(constraints, record, bandwidth, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint):
    # Set for storing already processed brands 
    brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    number = 0
    oilbases_dict = dict()

    for i in bandwidth['origin'].unique():
        oilbases_dict[i] = number
        number+=1
        
    origins_with_inf_supply = set(oilbases_dict.values()).difference(constants['ORIGINS_TO_CONTROL'])
    
    for b in brands:
        if b in brands_accounted_for:
            continue
        
        brands_overlap_numbers = merge_supply_by_product(b, match_brand_product, match_brand_number, match_product_brand, match_number_brand)
        brands_accounted_for.update(brands_overlap_numbers)

        brands_list = [brand for brand in record.values() if brand.get('id') in brands_overlap_numbers]

        origins = []
        for brand in brands_list:
            origins += extract_unique_origins_of_brand(brand)
            
        origins = list(set(origins))
        
        transportations_list = []
        for brand in brands_list:
            transportations_list += get_brand_transportations(brand)

        for o in origins:
            
            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]
            product_name = match_brand_product[match_brand_number[b]]
            const = float(supply[(supply['origin'] == o) & (supply['product'] == product_name)]['supply'])
            
            if o in origins_with_inf_supply:
                continue
                
            else:

                # Add a restriction on supply >= lower_const
                const *= constants['PERCENT_LOADING']
                transportation_x = [x[transportation.get("var_name")] for transportation in origin_transportations]
                constraints.append(cp.sum(transportation_x) >= const)
                n_constraint +=1

    return constraints, n_constraint, x

In [42]:
# Make restriction on supply by sum of all products - lower bound
def make_restriction_supply_by_all_products_lower(constraints, record, supply, x, brands, n_constraint):
    
    number = 0
    oilbases_dict = dict()

    for i in bandwidth['origin'].unique():
        oilbases_dict[i] = number
        number+=1
        
    origins_with_inf_supply = set(oilbases_dict.values()).difference(constants['ORIGINS_TO_CONTROL'])
    
    brands_list = [brand for brand in record.values()]
    origins = []
    
    for brand in brands_list:
        origins += extract_unique_origins_of_brand(brand)
            
    origins = list(set(origins))
        
    for o in origins:
        if o in origins_with_inf_supply:
            continue

        else:
            transportations_list = []
            for brand in brands_list:
                transportations_list += get_brand_transportations(brand)            
            
                            
            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]

            const = float(supply[(supply['origin'] == o)]['supply'].sum())
            
            # Add a restriction on supply >= lower_const
            const *= constants['PERCENT_LOADING']
            transportation_x = [x[transportation.get("var_name")] for transportation in origin_transportations]
            constraints.append(cp.sum(transportation_x) >= const)
            n_constraint +=1
    
    return constraints, n_constraint, x

#### Define restriction #3: all brands for one point should come from the same origin

In [43]:
def create_dummy_Name(row):
    return 'y_P' + str(int(row["point"])) + 'O' + str(int(row["origin"])), int(row["point"]), int(row["origin"])

In [44]:
def create_dummy(df):
    
    [*o] = df.apply(lambda row: create_dummy_Name(row), axis=1)
    df_dummy = pd.DataFrame(o, columns=['dummy_Name', 'point', 'origin'])
    df_dummy = df_dummy.reset_index()
    df_dummy = df_dummy.drop_duplicates(subset=['dummy_Name', 'point', 'origin'])    
    df_dummy.reset_index(inplace=True)
    df_dummy.drop(columns=['index', 'level_0'], inplace=True)
    
    # Create dummies AFTER wrong indexes are deleted...
    dummies_vars = cp.Variable(len(df_dummy), boolean=True)
    
    return df_dummy, dummies_vars

In [45]:
df_dummy, dummies_vars = create_dummy(df_reorganized)

In [48]:
def make_restriction_dummy(record, x, constraints, n_constraint, points_dict, dummies_vars, df_dummy):
    
    transportations = get_transportations(record)

    for p in points_dict.keys():
        
        for o in points_dict[p]:
            
            po_transportations = [transportation for transportation in transportations if transportation.get('origin') == o and transportation.get('point') == p]
            
            df_d = df_dummy[(df_dummy['point'] == p) & (df_dummy['origin'] == o) ]
            
            # Restriction (...) <= M * Y_PnOm       
            transportations_x = [x[transportation.get('var_name')] for transportation in po_transportations]
            constraints.append(cp.sum(transportations_x) <= constants['M'] * dummies_vars[df_d.index])
            n_constraint += 1
            
    for p in points_dict.keys():
        
        df_d = df_dummy[df_dummy['point'] == p]
        # Restriction (y_PnOm1 + y_PnOm2 + ...) == 1 
        constraints.append(cp.sum(dummies_vars[df_d.index]) == 1)
        n_constraint += 1
    
    return constraints, n_constraint

In [49]:
# Create points - origins dictionary
def create_points_dict(df):
    points_dict = {}
    for p in list(df['point'].unique()):
        origins_available = list(df[df['point'] == p]['origin'].unique())
        if points_dict.get(p) == None:
            points_dict[p] = origins_available
    return points_dict

points_dict = create_points_dict(df_reorganized)

In [50]:
cbc_start = time.time()

x, objective = define_problem(df_reorganized)

constraints, n_constraint, x = make_restriction_demand(brand_info_map, brands, x, demand)

if constants['UPPER_BOUND'] == 'group':
    constraints, n_constraint, x = make_restriction_supply_by_group_upper(constraints, brand_info_map, bandwidth_subset, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint)
elif constants['UPPER_BOUND'] == 'product':
    constraints, n_constraint, x = make_restriction_supply_by_product_upper(constraints, brand_info_map, bandwidth_subset, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint)

if constants['LOWER_BOUND'] == 'group':
    constraints, n_constraint, x = make_restriction_supply_by_group_lower(constraints, brand_info_map, bandwidth_subset, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint)
elif constants['LOWER_BOUND'] == 'product':
    constraints, n_constraint, x = make_restriction_supply_by_product_lower(constraints, brand_info_map, bandwidth_subset, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint)
elif constants['LOWER_BOUND'] == 'all producs':
    make_restriction_supply_by_all_products_lower(constraints, brand_info_map, supply, x, brands, n_constraint)
    
constraints, n_constraint = make_restriction_dummy(brand_info_map, x, constraints, n_constraint, points_dict, dummies_vars, df_dummy)

cbc_end = time.time()

# Time for restrictions insertion
cbc_end - cbc_start

315.3352372646332

In [ ]:
def main(problem):
    cbc_start = time.time()
    problem = cp.Problem(objective, constraints)
    solver= problem.solve(cp.CBC, verbose=True, maximumSeconds=1800)
    cbc_end = time.time()
    print(cbc_end - cbc_start)

In [ ]:
if __name__ == '__main__':
    main(problem)

Below the results of model are represented

In [51]:
cbc_start = time.time()

In [52]:
problem = cp.Problem(objective, constraints)
solver= problem.solve(cp.CBC, verbose=True, maximumSeconds=1800)

                                     CVXPY                                     
                                     v1.3.0                                    
(CVXPY) Jun 01 10:49:06 AM: Your problem has 67894 variables, 20742 constraints, and 0 parameters.
(CVXPY) Jun 01 10:49:11 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 01 10:49:11 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 01 10:49:11 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 01 10:49:18 AM: Compiling problem (target solver=CBC).
(CVXPY) Jun 01 10:49:18 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuff

D:\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [53]:
cbc_end = time.time()

In [54]:
# Time spent for model building solution
(cbc_end - cbc_start)

2306.140187740326